<h1>Job Market Trends</h1>
<h2>Extract, Transform, and Load Data</h2>

Data Analyst vs Data Scientist job

In [14]:
import os
import codecs
from bs4 import BeautifulSoup
import csv

from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, Text, insert, select, delete

<h2>Part 1: Access data files within a Directory</h2>

The job postings are stored as files within a directory, so we will create a function to iterate through files in a directory to be able to open each one.

In [15]:
# print a list of the files in the working directory
print(os.getcwd())
#!ls

/Users/jennifer/nlp-jobmarket
1A main_etl_analyst_csv.ipynb         README.md
1B main_etl_analyst_sql-Copy1.ipynb   joblist.sqlite
1B main_etl_analyst_sql.ipynb         main_etl_scientist_sql.py
1B main_etl_scientist_sql.ipynb       main_jobdesc_eda.ipynb
2A main_csv_jobdesc_nlp_preproc.ipynb results.csv
2B main_sql_jobdesc_nlp_preproc.ipynb test_folder
Data Analyst                          test_folder2
Data Scientist


In [16]:
directory = "test_folder2"
fileList = []

def get_raw_data(directory):
    '''Open file containing html of job description and prepare soup object.'''
    fileList = []
    soupList = []
    # Iterate through each file in directory
    for file in os.listdir(directory):
        if file.endswith(".txt"):
            # add each filename to list
            fileList.append(file)
            #print(fileList)
            # open and load html
            with codecs.open(directory + "/"+ file, 'r', "utf-8") as f:
                job_html = f.read()
                job_soup = BeautifulSoup(job_html, "html.parser")
                soupList.append(job_soup)
    return soupList

In [17]:
soupList = get_raw_data("test_folder2")
#print(soupList)

Great. We are able to open each of the .txt files that are in our directory of interest.

<h2>Part 2 : Opening and extracting information from files</h2>

First, we will use a test directory with six files to test to make sure we can pull out the information we want. Some companies have ratings available and some do not. This affects how data can be parsed using the various tags.

- Untitled 14-22-33.txt
- Untitled 14-36-26.txt -- TROX
- Untitled 14-41-46 -- KILLI
- Untitled 14-45-32 -- Citi worked
- Untitled 14-25-49 -- CIBC worked
- Untitled 14-19-29 -- TalentSphere, all worked incl salary

In [18]:
# Extract information from one file to test code
with codecs.open("test_folder2/Untitled 14-19-29.txt", 'r', "utf-8") as f:
    job_html = f.read()
job_soup = BeautifulSoup(job_html, "html.parser")

#print(job_soup)

In [19]:
job_title = job_soup.find("h1").text.strip()
print(job_title)

Financial Data Analyst


In [20]:
# The company name could be extracted from a few places.
# Tried a number of different options to test which worked for the most number of postings
try:
    company = job_soup.find("div", class_="jobsearch-InlineCompanyRating").text
    print('try 1: ', company)
except:
    pass

try:
    company = job_soup.find("div", class_="jobsearch-InlineCompanyRating").next_element.text.strip()
    print('try 2: ', company)
except:
    pass

try:
    company = job_soup.find("div", class_="jobsearch-CompanyReview--heading").text.strip()
    print('try 3: ', company)
except:
    pass

try:
    company = job_soup.find("div", class_="jobsearch-InlineCompanyRating").next_element.next_element.text.strip()
    print('try 4: ', company)
except:
    pass

# some times contains number of reviews
try:
    job_location = job_soup.find("div", class_ = "jobsearch-InlineCompanyRating").text
    print('try 5: ', company)
except:
    pass

try 1:  

TalentSphere Staffing Solutions


14 reviews
try 3:  TalentSphere Staffing Solutions
try 4:  TalentSphere Staffing Solutions
try 5:  TalentSphere Staffing Solutions


In [21]:
# using .next_element give AttributeError: 'NavigableString' object has no attribute 'text'
# job_location = job_soup.find("div", class_ = "jobsearch-InlineCompanyRating").next_element.text

for sibling in job_soup.find("div", class_ = "jobsearch-InlineCompanyRating").next_siblings:
    print(repr(sibling))

<div>Toronto, ON</div>


In [22]:
job_location = job_soup.find("div", class_ = "jobsearch-InlineCompanyRating").next_sibling.next_sibling.text.strip()
print(job_location)

AttributeError: 'NoneType' object has no attribute 'text'

In [23]:
# One method doesn't seem to work for all the files in the test directory.
# use a try-except block

try:
    job_location = job_soup.find("div", class_ = "jobsearch-InlineCompanyRating").next_sibling.text.strip()
    print('try-block worked: ', job_location)
except:
    try:
        job_location = job_soup.find("div", class_ = "jobsearch-InlineCompanyRating").next_sibling.next_sibling.text.strip()
        print('location except-block -- ', job_location)
    except:
        job_location = 'NaN'

try-block worked:  Toronto, ON


In [24]:
try:
    job_salary = job_soup.find("span", class_="icl-u-xs-mr--xs").text.strip()
except AttributeError:
    job_salary = "NaN"
print(job_salary)

$50,000 - $60,000 a year


In [25]:
job_description = job_soup.find("div", class_="jobsearch-jobDescriptionText").text.strip().replace('\n', ' ')
print(job_description)

Job Title: Financial Data Analyst Industry Sector: Nor for Profit Salary: $50,000-$60,000 + great benefits, vacation and w/l balance. Location: Toronto – currently WFH  Our vey busy not for profit client is seeking a Financial Data Analyst to join their small, dynamic team to work on financial modelling and statistical research analysis.  About our client This NPO client is in the philanthropic space has been improving the lives of present and future generations through Education, Research, Healthcare and the Arts for over 30 years. They continue to grow its foundation of members, organizations and programs.  About the role Working closely with the other members of the finance team, and reporting directly into the Controller, responsibilities will include but not be limited to: Perform financial forecasting, reporting, and metrics tracking. Analyze financial data and create financial models for decision support. Report on financial performance and prepare for regular leadership reviews

In [26]:
print(len(job_description))

2441


In [27]:
job_record = {'jobtitle': job_title,
              'company': company,
              'location': job_location,
              'salary': job_salary,
              'jobdescription': job_description,
              'label': 0
             }
print(job_record)

{'jobtitle': 'Financial Data Analyst', 'company': 'TalentSphere Staffing Solutions', 'location': 'Toronto, ON', 'salary': '$50,000 - $60,000 a year', 'jobdescription': 'Job Title: Financial Data Analyst Industry Sector: Nor for Profit Salary: $50,000-$60,000 + great benefits, vacation and w/l balance. Location: Toronto – currently WFH  Our vey busy not for profit client is seeking a Financial Data Analyst to join their small, dynamic team to work on financial modelling and statistical research analysis.  About our client This NPO client is in the philanthropic space has been improving the lives of present and future generations through Education, Research, Healthcare and the Arts for over 30 years. They continue to grow its foundation of members, organizations and programs.  About the role Working closely with the other members of the finance team, and reporting directly into the Controller, responsibilities will include but not be limited to: Perform financial forecasting, reporting, 

To persist the data, we will save the records directly a SQlite database.

In [15]:
# Import necessary module
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, Text

# Create engine: engine
engine = create_engine('sqlite:///joblist.sqlite')
metadata = MetaData()

# Define a new table
data = Table('data', metadata,
             Column('jobtitle', String(100)),
             Column('company', String(100)),
             Column('location', String(25)),
             Column('salary', Integer()),
             Column('jobdescription', Text()),
             Column('label', Integer())
)

# Create table
metadata.create_all(engine)

# Print table details
print(engine.table_names())

['data']


In [16]:
# Import insert and select from sqlalchemy
from sqlalchemy import insert, select

# Build an insert statement to insert a record into the data table: insert_stmt
insert_stmt = insert(data).values(jobtitle= job_title,
                                 company= company,
                                 location= job_location,
                                 salary= job_salary,
                                 jobdescription= job_description,
                                 label= 0
                                 )

# Execute the insert statement via the connection: results
connection = engine.connect()
results = connection.execute(insert_stmt)

# Print result rowcount
print("The number of rows added is: ", results.rowcount)

The number of rows added is:  1


Verify that we added something to our new database.

In [17]:
# SELECT the row we just added to db
stmt = 'SELECT * from data'
results = connection.execute(stmt).fetchall()
print(results)

[('Financial Data Analyst', 'TalentSphere Staffing Solutions', 'Toronto, ON', '$50,000 - $60,000 a year', 'Job Title: Financial Data Analyst\nIndustry Sector: Nor for Profit\nSalary: $50,000-$60,000 + great benefits, vacation and w/l balance.\nLocation: To ... (2175 characters truncated) ...  minded, intelligent, fun, dynamic people who love the company they work in and all it stands for! What could be better than that?!\n\nTSSHP\n#LI-TS1', 0)]


In [ ]:
# The following stmt is the same as above
# stmt = select([data])

Remove this row so database is empty.

In [18]:
from sqlalchemy import delete

# DELETE row from database
delete_stmt = delete(data)
results = connection.execute(delete_stmt)
print("The number of rows is: ", results.rowcount)

The number of rows is:  1


In [20]:
# Check to see that data table in db was properly deleted

# Print table details
print('List of tables:', engine.table_names())

List of tables: ['data']


In [21]:
# SELECT the row we just added to db
stmt = 'SELECT * from data'
results = connection.execute(stmt).fetchall()
print(results)

[]


In [ ]:
# Close connection
#connection.close()

<h2>Part 3 : Put it all together</h2>

Put all the steps together so that we can easily extract job information from each text file and keep a record of which files we have opened.

In [1]:
import os
import codecs
from bs4 import BeautifulSoup
import csv

from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, Text, insert, select, delete

def get_raw_data(directory):
    '''Open file containing html of job description and prepare soup object.'''
    fileList = []
    soupList = []
    # Iterate through each file in directory
    for file in os.listdir(directory):
        if file.endswith(".txt"):
            # add each filename to list
            fileList.append(file)
            #print(fileList)
            # open and load html
            with codecs.open(directory + "/"+ file, 'r', "utf-8") as f:
                job_html = f.read()
                job_soup = BeautifulSoup(job_html, "html.parser")
                soupList.append(job_soup)
    print("soup_list is done.")
    return soupList

# From the loaded text, extract job information using beautiful soup
def get_job_record(job_soup):
    '''Create a record of information for one job.'''
    # Title
    try:
        job_title = job_soup.find('h1').text.strip()
    except:
        job_title = "NaN"
    
    # Company
    try:
        company = job_soup.find("div", class_="jobsearch-InlineCompanyRating").next_element.next_element.text.strip()
    except:    
        try:
            company = job_soup.find("div", class_="jobsearch-InlineCompanyRating").text.strip()
        except:
            company = "NaN"

    # Location
    try:
        job_location = job_soup.find("div", class_ = "jobsearch-InlineCompanyRating").next_sibling.text.strip()
    except:
        try:
            job_location = job_soup.find("div", class_ = "jobsearch-InlineCompanyRating").next_sibling.next_sibling.text.strip()
        except:
            job_location = 'NaN'

    # Job Description
    try:
        job_description = job_soup.find("div", class_="jobsearch-jobDescriptionText").text.strip().replace('\n', ' ')
    except:
        job_description = "NaN"

    # Not all postings have a salary available
    try:
        job_salary = job_soup.find("span", class_="icl-u-xs-mr--xs").text.strip()
    except AttributeError:
        job_salary = "NaN"
    
    job_record = {'jobtitle': job_title,
                                 'company': company,
                                 'location': job_location,
                                 'salary': job_salary,
                                 'jobdescription': job_description,
                                 'label': 0
                 }

    return job_record

def main_etl(directory):
    '''This function loads text data, extracts pertinent job information, and saves data in a sql database.'''
    soupList = get_raw_data(directory)
        
    # add each job record to a list
    # this will create a list of dictionaries, making it easy to insert into a sql table
    job_records = []
    for job_soup in soupList:
        job_record = get_job_record(job_soup)
        job_records.append(job_record)
        #print("Added to job_records list. Length of job_records is: ", len(job_records))

    # add job records to sqlite db
    # Create engine: engine
    engine = create_engine('sqlite:///joblist.sqlite')
    metadata = MetaData()

    # Define a new table
    data = Table('data', metadata,
                 Column('jobtitle', String(100)),
                 Column('company', String(100)),
                 Column('location', String(25)),
                 Column('salary', Integer()),
                 Column('jobdescription', Text()),
                 Column('label', Integer())
                )

    # Create table
    metadata.create_all(engine)

    # Print table details
    print(engine.table_names())

    # Build an insert statement to insert a record into the data table: insert_stmt
    insert_stmt = insert(data)

    # Execute the insert statement via the connection: results
    connection = engine.connect()
    results = connection.execute(insert_stmt, job_records)

    # Print result rowcount
    #print("The number of rows added is: ", results.rowcount)

Let's test out the functionality on another folder containing files with job description in html format.

In [2]:
print(os.getcwd())

/Users/jennifer/nlp-jobmarket


In [3]:
!ls

1A main_etl_analyst_csv.ipynb         README.md
1B main_etl_analyst_sql.ipynb         joblist.sqlite
1B main_etl_scientist_sql.ipynb       main_etl_scientist_sql.py
2A main_csv_jobdesc_nlp_preproc.ipynb main_jobdesc_eda.ipynb
2B main_sql_jobdesc_nlp_preproc.ipynb results.csv
Data Analyst                          test_folder
Data Scientist                        test_folder2


In [7]:
from sqlalchemy import create_engine, delete, func

engine = create_engine('sqlite:///joblist.sqlite')
connect = engine.connect()

print(engine.table_names())

['data']


In [8]:
# Load in data table
metadata = MetaData()

data = Table('data', metadata, autoload=True, autoload_with=engine)

print(data.columns.keys())
print(repr(metadata.tables['data']))

['jobtitle', 'company', 'location', 'salary', 'jobdescription', 'label']
Table('data', MetaData(bind=None), Column('jobtitle', VARCHAR(length=100), table=<data>), Column('company', VARCHAR(length=100), table=<data>), Column('location', VARCHAR(length=25), table=<data>), Column('salary', INTEGER(), table=<data>), Column('jobdescription', TEXT(), table=<data>), Column('label', INTEGER(), table=<data>), schema=None)


In [9]:
# Count the number of rows
# Should be 0
stmt = select([func.count(data.columns.jobdescription)])
connect.execute(stmt).scalar()

0

In [10]:
dataAnalyst = main_etl("Data Analyst")

soup_list is done.
Added to job_records list. Length of job_records is:  1
Added to job_records list. Length of job_records is:  2
Added to job_records list. Length of job_records is:  3
Added to job_records list. Length of job_records is:  4
Added to job_records list. Length of job_records is:  5
Added to job_records list. Length of job_records is:  6
Added to job_records list. Length of job_records is:  7
Added to job_records list. Length of job_records is:  8
Added to job_records list. Length of job_records is:  9
Added to job_records list. Length of job_records is:  10
Added to job_records list. Length of job_records is:  11
Added to job_records list. Length of job_records is:  12
Added to job_records list. Length of job_records is:  13
Added to job_records list. Length of job_records is:  14
Added to job_records list. Length of job_records is:  15
Added to job_records list. Length of job_records is:  16
Added to job_records list. Length of job_records is:  17
Added to job_records 

Added to job_records list. Length of job_records is:  284
Added to job_records list. Length of job_records is:  285
Added to job_records list. Length of job_records is:  286
Added to job_records list. Length of job_records is:  287
Added to job_records list. Length of job_records is:  288
Added to job_records list. Length of job_records is:  289
Added to job_records list. Length of job_records is:  290
Added to job_records list. Length of job_records is:  291
Added to job_records list. Length of job_records is:  292
Added to job_records list. Length of job_records is:  293
Added to job_records list. Length of job_records is:  294
Added to job_records list. Length of job_records is:  295
Added to job_records list. Length of job_records is:  296
Added to job_records list. Length of job_records is:  297
Added to job_records list. Length of job_records is:  298
Added to job_records list. Length of job_records is:  299
Added to job_records list. Length of job_records is:  300
Added to job_r

In [11]:
!ls

1A main_etl_analyst_csv.ipynb         README.md
1B main_etl_analyst_sql.ipynb         joblist.sqlite
1B main_etl_scientist_sql.ipynb       main_etl_scientist_sql.py
2A main_csv_jobdesc_nlp_preproc.ipynb main_jobdesc_eda.ipynb
2B main_sql_jobdesc_nlp_preproc.ipynb results.csv
Data Analyst                          test_folder
Data Scientist                        test_folder2
